In [1]:
# all imports
import json
import numpy as np
import os
import cv2
import io
import dataSetCreator as dc
import glob
import gc
import random
import shutil

## Parameters

In [2]:
# path to warning signs files

signs_path="/home/tiffany/master_projekt/dataset_collection/FinalDataSet/"
backgrounds_path = "/home/tiffany/master_projekt/dataset_collection/FinalDataSetCombined/"
#backgrounds_path = "/media/tiffany/INTENSO/FinalDataSetCombined/"

#signs_path="/home/tiffany/master_projekt/dataset_collection/TestDataSet/"
#backgrounds_path = "/media/tiffany/INTENSO/TestDataSetCombined/"

# load in background file for later
#background_json_path = "/home/tiffany/master_projekt/dataset_collection/backgrounds/"
#backgrounds_file = "backgrounds_test.json"
background_json_path = "/home/tiffany/master_projekt/dataset_collection/WBackgrounds/"
backgrounds_file = "backgrounds.json"

if os.path.exists(background_json_path+backgrounds_file):
    print("File found and loaded into data variable")
    with open(background_json_path+backgrounds_file) as f:
        data = json.load(f)
else:
    print("No file found")
    
print("Data Size:", len(data))
print(data)

File found and loaded into data variable
Data Size: 37
{'background_1.jpg': {'height': 720, 'img_name': 'background_1.jpg', 'img_path': '/home/tiffany/master_projekt/dataset_collection/WBackgrounds/', 'positions': [[508, 212], [546, 199], [512, 257], [549, 248]], 'positions2': [[508, 212], [546, 199], [549, 248], [512, 257]], 'shape': [720, 960, 3], 'width': 960}, 'background_10.jpg': {'height': 720, 'img_name': 'background_10.jpg', 'img_path': '/home/tiffany/master_projekt/dataset_collection/WBackgrounds/', 'positions': [[534, 58], [550, 47], [533, 147], [547, 147]], 'positions2': [[534, 58], [550, 47], [547, 147], [533, 147]], 'shape': [720, 960, 3], 'width': 960}, 'background_11.jpg': {'height': 720, 'img_name': 'background_11.jpg', 'img_path': '/home/tiffany/master_projekt/dataset_collection/WBackgrounds/', 'positions': [[625, 15], [668, 6], [623, 108], [659, 108]], 'positions2': [[625, 15], [668, 6], [659, 108], [623, 108]], 'shape': [720, 960, 3], 'width': 960}, 'background_12.jp

In [3]:
path=os.path.dirname(signs_path)
os.path.basename(path)

'FinalDataSet'

In [4]:
print ("Collecting all folders")
dirlist = [ item for item in os.listdir(signs_path) if os.path.isdir(os.path.join(signs_path, item)) ]
print (dirlist)
print ("Folders collected")

for folder in dirlist:
    print ("#############################",folder, ":")
    sign = folder.split("Warning")
    file_counter = 1
    current_path = backgrounds_path+folder+"/"
    if not os.path.exists(current_path):
        os.mkdir(current_path)
    for file in os.listdir(signs_path+folder):
        # grab image
        img = cv2.imread(signs_path+folder+"/"+file)
        print("IMAGE: ", file)
        # list of image rotated
        rotated = dc.rotateImage(signs_path+folder+"/", file, 7)
        print("rotated size:", len(rotated))
        red_img = []
        invert_img = []
        blur = []
        blur_kernels = [15,23,27,35] # 3,5,7,9,13 normally
        endings = (".jpg", ".png")

        # coloring every single rotated image
        for index in range(len(rotated)):
            red_img.append(dc.color_image_red(rotated[index].copy()))
            invert_img.append(dc.invert_image_color(rotated[index].copy()))
            for i in range(len(blur_kernels)):
                blur.append(dc.gaussian_filter(rotated[index].copy(), blur_kernels[i]))

        combined_path = current_path+"combined_"+str(file_counter)+"/"
        background_folder = "background_"
        #all_images = rotated + red_img + invert_img + blur
        all_images = rotated + red_img + invert_img + blur
        
        del blur
        del invert_img
        del rotated
        del red_img
        gc.collect()
        
        print("All images size", len(all_images))
        if not os.path.exists(combined_path):
            os.mkdir(combined_path)      
            for i in range(1,len(data)+1):
                n_img = []
                single_bg_path = combined_path+background_folder+str(i)+"/"
                if not os.path.exists(single_bg_path):
                    os.mkdir(single_bg_path)
                for l in range (len(all_images)):
                    h = dc.HomoPerspective(data[background_folder+str(i)+".jpg"])
                    h.passImage(all_images[l])
                    n_img.append(h.combineImages())
                img_name = sign[1]+"_"+str(file_counter)+"_"+background_folder+"_"+str(i)
                dc.saveImages(img_name, single_bg_path, n_img)
                del h
                del n_img
        del all_images
        del img
        gc.collect()
        file_counter+=1
    os.system('spd-say "Images were created for '+sign[1]+'"')
print("############# DONE! #################")

['WarningBiohazard']
Folders collected
############################# WarningBiohazard :
IMAGE:  2000px-ISO_7010_W009.svg.png
rotated size: 140
All images size 980
IMAGE:  44-447513_biohazard-transparent-triangle-warning-signs-biohazard-hd-png.png
rotated size: 140
All images size 980
IMAGE:  6993062.jpg
rotated size: 140
All images size 980
IMAGE:  45014_warnzeichen_aufkleber_schild_warning_biohazard.jpg
rotated size: 140
All images size 980
IMAGE:  depositphotos_14710269-stock-photo-warning-biohazard-sign.jpg
rotated size: 140
All images size 980
############# DONE! #################


In [30]:
for folder in dirlist:
    print ("#############################",folder, ":")
    sign = folder.split("Warning")
    print (sign)

############################# WarningBiohazard :
['', 'Biohazard']
############################# WarningCorrosive :
['', 'Corrosive']


In [ ]:
rotated = dc.rotateImage(signs_path+"WarningGeneral"+"/", "500_F_111301924_gIurLLhD7LuelJgqi4L4DTPQTvEZDalk.jpg", 7)
print(len(rotated))
gaus = dc.gaussian_filter(rotated[89].copy(), 15)
h = dc.HomoPerspective(data["background_"+str(2)+".jpg"])
h.passImage(gaus)
pic = h.combineImages()
#cv2.imwrite('final.jpg', final)
cv2.imshow('FINAL', pic)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
signs_path="/home/tiffany/master_projekt/dataset_collection/FinalDataSetCombined/"
save_path = "/media/tiffany/Lexar/49_backgrounds/background_"

In [3]:
# go to warning file
# go to combined_x
# create random numbers between 1-140, 141-280, 281-420, 421-980...
# use random numbers to take files in
# go to combined_x...
# path to warning signs files

print ("Collecting all folders: ")
dirlist = [ item for item in os.listdir(signs_path) if os.path.isdir(os.path.join(signs_path, item)) ]
print (dirlist)

for i in range(1,50):
    current_background = "background_"+str(i)
    for folder in dirlist:
        current_path = signs_path+folder+"/"
        combined_list = [ item for item in os.listdir(current_path) if os.path.isdir(os.path.join(current_path, item)) ]
        #print(combined_list)
        img_list = []
        sign = folder.split("Warning")
        for combined_folder in combined_list:
            normal_range = random.randint(4,12)
            red_range = random.randint(3,6)
            inverted_range = random.randint(3,6)
            rotated_range = random.randint(4,12)
            i1_list = [random.randint(1, 140) for p in range(normal_range)] #normal
            i2_list = [random.randint(141, 280) for p in range(red_range)] #red
            i3_list = [random.randint(281, 420) for p in range(inverted_range)] #inverted
            i4_list = [random.randint(421, 980) for p in range(rotated_range)] #rotated
            combined_nr = combined_folder.split("_")
            print(combined_nr)
            combined_path = current_path+combined_folder+"/"+current_background+"/"
            #print(combined_path)
            if os.path.exists(combined_path):
                #imgs_list = [ item for item in os.listdir(combined_path) ]
                #print(os.listdir(combined_path)[0])
                dest_path = save_path+str(i)+"/"
                for i1 in i1_list:
                    img_name = sign[1]+"_"+str(combined_nr[1])+"_"+"background__"+str(i)+"_"+str(i1)+".jpg"
                    img_path = combined_path+img_name
                    if os.path.isfile(img_path):
                        shutil.copy(img_path, dest_path)
                    del img_path
                for i2 in i2_list:
                    img_name = sign[1]+"_"+str(combined_nr[1])+"_"+"background__"+str(i)+"_"+str(i2)+".jpg"
                    img_path = combined_path+img_name
                    if os.path.isfile(img_path):
                        shutil.copy(img_path, dest_path)
                    del img_path
                for i3 in i3_list:
                    img_name = sign[1]+"_"+str(combined_nr[1])+"_"+"background__"+str(i)+"_"+str(i3)+".jpg"
                    img_path = combined_path+img_name
                    if os.path.isfile(img_path):
                        shutil.copy(img_path, dest_path)
                    del img_path
                for i4 in i4_list:
                    img_name = sign[1]+"_"+str(combined_nr[1])+"_"+"background__"+str(i)+"_"+str(i4)+".jpg"
                    img_path = combined_path+img_name
                    if os.path.isfile(img_path):
                        shutil.copy(img_path, dest_path)
                    del img_path
            del normal_range
            del red_range
            del inverted_range
            del rotated_range
            del i1_list
            del i2_list
            del i3_list
            del i4_list


['WarningGeneral']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combined', '2']
['combined', '3']
['combined', '5']
['combined', '7']
['combined', '4']
['combined', '6']
['combined', '1']
['combine

In [10]:
x = [random.randint(1, 140) for p in range(7)]
print (x)

[129, 57, 56, 111, 89, 45, 87]


In [15]:
random.randint(1,10)

7

In [5]:
## putting all data from background folders into train first
path = "/media/tiffany/Lexar/37_backgrounds/"
save_path = "/home/tiffany/Desktop/object-detection-final/images/train/"
dirlist = [ item for item in os.listdir(path) if os.path.isdir(os.path.join(path, item)) ]
print(dirlist)

for i in range(1,17):
    current_path = path+"background_"+str(i)+"/"
    for file in os.listdir(current_path):
        shutil.copy(current_path+file, save_path)
os.system('spd-say "All files were successfully copied into train folder" ')
print ("DONE!")

['background_1', 'background_2', 'background_3', 'background_4', 'background_5', 'background_6', 'background_7', 'background_8', 'background_9', 'background_10', 'background_11', 'background_12', 'background_13', 'background_14', 'background_15', 'background_16', 'background_17', 'background_18', 'background_19', 'background_20', 'background_21', 'background_22', 'background_23', 'background_24', 'background_25', 'background_26', 'background_27', 'background_28', 'background_29', 'background_30', 'background_31', 'background_32', 'background_33', 'background_34', 'background_35', 'background_36', 'background_37']
DONE!
